To run this notebook you need first to install the kif-llm-sotre package in your virtual environment. Run `pip install -e .` in your terminal. 

In [1]:
import os
# Use to get you environment variables
import dotenv
dotenv.load_dotenv()

# import kif-lib
from kif_lib import *
from kif_lib.vocabulary import wd

# import our KIF-LLM-Store extension
from kif_llm_store import *
from kif_llm_store.store.llm.constants import LLM_Providers, EntityResolutionMethod

# uncomment to print execution information (including LLM prompts)
# import logging
# logging.basicConfig(level=logging.INFO)

In [2]:
# instanciate a LLM Store
kb: LLM_Store = Store(
  LLM_Store.store_name, # indicate it is an LLM Store
  llm_provider=LLM_Providers.IBM, # We are using IBM WatsonX AI Models
  model_id='meta-llama/llama-3-2-1b-instruct', # the model id we are using
  base_url=os.environ['LLM_API_ENDPOINT'], #endpoint to the llm models
  api_key=os.environ['LLM_API_KEY'], # api key to access those models
  project_id=os.environ['WATSONX_PROJECT_ID'],
  model_params= { 'decoding_method': 'greedy' }
)

### Brazil official language

In [3]:
# define a pattern to query llm store
subject = wd.Brazil # or wd.Q(155, 'Brazil')
property = wd.official_language # or wd.P(37, 'official language')

# execute pattern on llm store and print the results:
result = kb.filter(subject=subject, property=property, distinct=False, annotated=False)
for stmt in result:
    display(stmt)

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [official language](http://www.wikidata.org/entity/P37)) (**Item** [Portuguese](http://www.wikidata.org/entity/Q5146))))

In [4]:
# User can define its own task prompt template that will override the LLM Store default filter compilation:
# A task prompt template must have at least one of the following placeholders: {subject}, {property}, and {value}
kb.task_prompt_template = 'What is the {property} of {subject}?'
result = kb.filter(subject=wd.Brazil, property=wd.official_language)
for stmt in result:
    display(stmt)

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [official language](http://www.wikidata.org/entity/P37)) (**Item** [Portuguese](http://www.wikidata.org/entity/Q5146))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [official language](http://www.wikidata.org/entity/P37)) (**Item** [Spanish](http://www.wikidata.org/entity/Q1321))))

In [5]:
# Use the default compiled task prompt template
kb.task_prompt_template = None

In [6]:
# execute pattern on llm store and print the results:
result = kb.filter(subject=wd.Brazil, property=wd.population)
for stmt in result:
    display(stmt)

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [population](http://www.wikidata.org/entity/P1082)) (**Quantity** 2.8)))

In [7]:
# countries that shares border with Brazil
result = kb.filter(subject=wd.Brazil, property=wd.shares_border_with, value=wd.instance_of(wd.country_))
for stmt in result:
    display(stmt)

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Uruguay](http://www.wikidata.org/entity/Q77))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Chile](http://www.wikidata.org/entity/Q298))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [South America](http://www.wikidata.org/entity/Q18))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

In [8]:
# countries that shares border with Brazil (Now Brazil is the value not the subject)
result = kb.filter(subject=wd.instance_of(wd.country_), property=wd.shares_border_with, value=wd.Brazil)
for stmt in result:
    display(stmt)

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

(**Statement** (**Item** [wd:Q83440](http://www.wikidata.org/entity/Q83440)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

In [9]:
# Try in advance to transform your filter into a natural language question (Testing feature – use with caution.)
kb.compile_to_natural_language_question = True

# countries that shares border with Brazil
result = kb.filter(subject=wd.Brazil, property=wd.shares_border_with, value=wd.instance_of(wd.country_))
for stmt in result:
    display(stmt)
display('-----------------------')
# countries that shares border with Brazil (Now Brazil is the value not the subject)
result = kb.filter(subject=wd.instance_of(wd.country_), property=wd.shares_border_with, value=wd.Brazil)
for stmt in result:
    display(stmt)

# Both filters should be transformed in: Which countries share a border with Brazil?

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Guyana](http://www.wikidata.org/entity/Q734))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Bolivia](http://www.wikidata.org/entity/Q750))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Peru](http://www.wikidata.org/entity/Q419))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Venezuela](http://www.wikidata.org/entity/Q717))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Colombia](http://www.wikidata.org/entity/Q739))))

...

(**Statement** (**Item** [Argentina](http://www.wikidata.org/entity/Q414)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

(**Statement** (**Item** [wd:Q736](http://www.wikidata.org/entity/Q736)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

(**Statement** (**Item** [Colombia](http://www.wikidata.org/entity/Q739)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

(**Statement** (**Item** [Peru](http://www.wikidata.org/entity/Q419)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

(**Statement** (**Item** [Venezuela](http://www.wikidata.org/entity/Q717)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Brazil](http://www.wikidata.org/entity/Q155))))

In [10]:
# Turn off the compilation of filters into natural language questions
kb.compile_to_natural_language_question = False

In [11]:
# execute pattern on llm store and print the results:
result = kb.filter(subject=wd.benzene, property=wd.mass)
for stmt in result:
    display(stmt)

(**Statement** (**Item** [benzene](http://www.wikidata.org/entity/Q2270)) (**ValueSnak** (**Property** [mass](http://www.wikidata.org/entity/P2067)) (**Quantity** 159)))